In [1]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
import requests, json, os

os.environ["YA_IAM_TOKEN"] = ""
os.environ["YA_FOLDER_ID"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [2]:
def yandex_translate(target_language,texts_list):
    
    body = {
    "targetLanguageCode": target_language,
    "texts": texts_list,
    "folderId": os.environ.get("YA_FOLDER_ID")
    }
    
    headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer {0}".format(os.environ.get('YA_IAM_TOKEN'))
    }
    
    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate',
                             json = body,
                             headers = headers)
    
    response_text = json.loads(response.text)
    
    return (response_text)

In [3]:
def translate_and_make_prompt(phrase):
    template = """
Based on the phrase, determine its specificity and detail in the range from 0 to 1; add block "Explanation" where you give a detailed explanations of why you gave such estimates of detail and specifity

For example, the phrase “good cop” will be vague, and the phrase “a policeman who always follows the letter of the law in his work”

----

Here is an example of the answer you should give to the phrase "The guys stood on the sidewalk and spun the hula hoop":

Specificity: 0.4
Detail: 0.3

Explanation:
The phrase "The guys stood on the sidewalk and spun the hula hoop" is somewhat specific in that it describes a particular activity ("spinning the hula hoop") being performed by a specific group ("the guys") in a particular location ("the sidewalk"). However, the phrase lacks detail in that it does not provide any information about the identity of "the guys", how they are spinning the hula hoop, or any other context surrounding the activity. Overall, the phrase is moderately specific but lacking in detail.

Here is another example of answer you should give to the phrase 

----

Specificity: 0.7
Detail: 0.7

Explanation:
The phrase provides some specific details such as the location ("dirty sidewalk near the shawarma tent"), the action being performed ("twisted a rainbow-colored hula hoop"), and the appearance of the people ("guys in shabby clothes out of size"). However, it still leaves some room for interpretation and imagination, as it does not specify the exact number of people or the precise location.

----

Here is another example of answer you should give to the phrase "Lyuba paints the fence, forgetting about the worries":

Specificity: 0.6
Detail: 0.3

Explanation:
The phrase "Lyuba paints the fence, forgetting about the worries" is quite specific in that it names a person ("Lyuba") and describes a single action ("painting the fence") they are performing. However, the phrase lacks detail in that it does not provide any information about the context of the painting or the nature of the worries that Lyuba is forgetting about. Overall, the phrase is quite specific but lacks detail.

----

Now give an answer for the phrase "{phrase}"
"""

    prompt = PromptTemplate(
        input_variables=["phrase"],
        template=template)
    translated_phrase = yandex_translate('en',[phrase])['translations'][0]['text']
    text = prompt.format(phrase=translated_phrase)
    return (text)

In [5]:
phrase = 'Мы долго ждали своей участи'

def make_openai_response(prompt):
    llm = OpenAI(temperature=0.5,max_tokens=1024)
    open_ai_response = llm(prompt)
    return (open_ai_response)

open_ai_response = make_openai_response(translate_and_make_prompt(phrase))
ya_translate_response = yandex_translate('ru',[open_ai_response])
print(ya_translate_response['translations'][0]['text'])


Специфичность: 0,6
Деталь: 0.4

Объяснение:
Фраза "Мы долгое время ждали своей судьбы" несколько специфична в том смысле, что она описывает конкретную деятельность ("ожидание"), выполняемую определенной группой ("мы") в течение определенного периода времени ("долгое время"). Однако этой фразе не хватает деталей в том смысле, что она не предоставляет никакой информации о личности "нас" или характере ожидаемой судьбы. В целом, фраза умеренно конкретна, но в ней не хватает деталей.


In [6]:
phrase = 'Временами Джон чувствовал жуткую досаду из-за того, что ему не удалось тогда достигнуть успеха'

def make_openai_response(prompt):
    llm = OpenAI(temperature=0.5,max_tokens=1024)
    open_ai_response = llm(prompt)
    return (open_ai_response)

open_ai_response = make_openai_response(translate_and_make_prompt(phrase))
ya_translate_response = yandex_translate('ru',[open_ai_response])
print(ya_translate_response['translations'][0]['text'])


Специфичность: 0,8
Детализация: 0.7

Объяснение:
Фраза "Временами Джон чувствовал себя ужасно раздраженным из-за того, что тогда он не смог добиться успеха" довольно специфична в том смысле, что она называет человека ("Джон") и описывает конкретную эмоцию ("раздражение"), которую он испытывает. Кроме того, в нем приводятся некоторые подробности о контексте эмоции ("временами") и источнике эмоции ("тогда не смог добиться успеха"). В целом, фраза довольно конкретна и содержит некоторые детали.


In [7]:
phrase = 'Временами Джон чувствовал жуткую досаду из-за того, что ему не удалось в 1987 году стать вице-президентом компании'

def make_openai_response(prompt):
    llm = OpenAI(temperature=0.5,max_tokens=1024)
    open_ai_response = llm(prompt)
    return (open_ai_response)

open_ai_response = make_openai_response(translate_and_make_prompt(phrase))
ya_translate_response = yandex_translate('ru',[open_ai_response])
print(ya_translate_response['translations'][0]['text'])


Специфичность: 0,9
Детализация: 0.7

Объяснение:
Фраза "Временами Джон чувствовал себя ужасно раздраженным из-за того, что ему не удалось стать вице-президентом компании в 1987 году" довольно специфична в том смысле, что в ней назван человек ("Джон"), подробно описана определенная эмоция ("ужасно раздражен") и описано определенное событие ("не удалось стал вице-президентом компании в 1987 году"). Эта фраза также содержит некоторые детали в том смысле, что она указывает, когда произошло событие и интенсивность эмоций Джона. Однако в этой фразе все еще не хватает некоторых деталей, поскольку она не содержит никакой информации о контексте события или причинах, по которым Джону не удалось стать вице-президентом. В целом, фраза довольно конкретна и содержит некоторые детали.
